# July 2024 Scenario - Limiting Hospitalizations

## Time = December 28th, 2021 (right around upswing of Omicron wave), Location = LA County

### Load dependencies

In [2]:
import pandas as pd

import pyciemss
import pyciemss.visuals.plots as plots
import pyciemss.visuals.vega as vega
import pyciemss.visuals.trajectories as trajectories
from pyciemss.integration_utils.result_processing import cdc_format

### Load model with vaccinations, variants, and hospitalizations

In [1]:
model1 = "https://raw.githubusercontent.com/ciemss/program-milestones/12-epi-ensemble-challenge/18-month-milestone/evaluation/Epi_Ensemble_Challenge/SEIRHD_vacc_var_petrinet.json"

### Gather and process data

In [3]:
data1 = "LA_County_Covid19_CSA_14day_hospitalization_table.csv"